In [13]:
"""
Only for the 1st three buildings
"""

# get train files
import glob

train_folders = ['CAR', 'UAH', 'UJITI', 'UJIUB']

files_names = []
n_files = 0
for building in train_folders:
    cur_file_names = glob.glob('train/' + building + '/log*')
    files_names.append(sorted(cur_file_names))
    n_files += len(cur_file_names)

print(files_names)
print('There are %d files' % n_files)

[['train/CAR/logfile_CAR_R1_S3.txt', 'train/CAR/logfile_CAR_R1_S3mini.txt', 'train/CAR/logfile_CAR_R2_S3.txt', 'train/CAR/logfile_CAR_R2_S4.txt'], ['train/UAH/logfile_UAH_R1_S3.txt', 'train/UAH/logfile_UAH_R1_S4.txt', 'train/UAH/logfile_UAH_R2_S3.txt', 'train/UAH/logfile_UAH_R2_S4.txt', 'train/UAH/logfile_UAH_R4_S3.txt', 'train/UAH/logfile_UAH_R4_S4.txt'], ['train/UJITI/logfile_UJITI_R1_NEXUS5.txt', 'train/UJITI/logfile_UJITI_R2_NEXUS5.txt'], ['train/UJIUB/logfile_UJIUB_R1n_S3.txt', 'train/UJIUB/logfile_UJIUB_R1r_S3.txt', 'train/UJIUB/logfile_UJIUB_R2n_S3.txt', 'train/UJIUB/logfile_UJIUB_R2r_S3.txt', 'train/UJIUB/logfile_UJIUB_R3_S3.txt']]
There are 17 files


In [15]:
# import files into dataframe
import numpy as np
import pandas as pd

track= []
for building in files_names:
    track.append([])
    for track_name in building:
        print(track_name)
        track[-1].append(pd.read_csv(track_name, sep=';', engine='c', names=range(11)))
        print(np.unique(track[-1][-1][0]))

train/CAR/logfile_CAR_R1_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/CAR/logfile_CAR_R1_S3mini.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'MAGN' 'POSI' 'SOUN' 'WIFI']
train/CAR/logfile_CAR_R2_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'PROX' 'SOUN'
 'WIFI']
train/CAR/logfile_CAR_R2_S4.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'HUMI' 'LIGH' 'MAGN' 'POSI' 'PRES' 'PROX'
 'SOUN' 'TEMP' 'WIFI']
train/UAH/logfile_UAH_R1_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/UAH/logfile_UAH_R1_S4.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'HUMI' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN'
 'TEMP' 'WIFI']
train/UAH/logfile_UAH_R2_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/UAH/logfile_UAH_R2_S4.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'HUMI' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN'
 'TEMP' 'WIFI']
train/UAH/logfile_UAH_R4_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/UAH/logf

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
# Find time limits per POSI id
def posi_time_limits(pos_array):
    # create start and end index time array
    pos_array['ind_begin_time'] = np.zeros((pos_array.shape[0]))
    pos_array['ind_end_time'] = np.zeros((pos_array.shape[0]))
    pos_array['ind_end_time'].iloc[-1] = 99999

    for i in range(1, pos_array.shape[0]):
        cur_split = (float(pos_array['Time'].iloc[i]) + float(pos_array['Time'].iloc[i-1])) / 2
        pos_array['ind_begin_time'].iat[i] = cur_split       
        pos_array['ind_end_time'].iat[i-1] = cur_split       
    return pos_array

pos = []
for i in range(len(track)):
    pos.append([])
    for j in range(len(track[i])):
        pos[i].append(track[i][j][[1, 2, 3, 4, 5, 6, 7]].iloc[track[i][j][0].values == 'POSI'])
        pos[i][j] = pos[i][j].dropna(axis=1)
        pos[i][j].columns = ['Time', 'index', 'Lat', 'Lon', 'FloorID', 'BuildingID']
        pos[i][j] = posi_time_limits(pos[i][j])
print(pos[0][0].head())

          Time  index          Lat       Lon  FloorID  BuildingID  \
8512    30.341    1.0  40.31347091  -3.48315      0.0        10.0   
13709   48.878    2.0  40.31332032  -3.48312      0.0        10.0   
22475   80.175    3.0  40.31328135  -3.48342      0.0        10.0   
27694   98.724    4.0  40.31331275  -3.48342      0.0        10.0   
38394  136.709    5.0  40.31333944  -3.48335      0.0        10.0   

       ind_begin_time  ind_end_time  
8512           0.0000       39.6095  
13709         39.6095       64.5265  
22475         64.5265       89.4495  
27694         89.4495      117.7165  
38394        117.7165      142.1475  


/usr/local/lib/python3.4/dist-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
# associate signals with POSI ids. Spliting at middle of the time between POSI measurements
for i in range(len(track)):
    for j in range(len(track[i])):
        track[i][j]['POSI_floor'] = np.zeros((track[i][j].shape[0])) * np.nan
        track[i][j]['POSI_building'] = np.zeros((track[i][j].shape[0])) * np.nan
        for k in range(pos[i][j].shape[0]):
            ind_begin_time = pos[i][j]['ind_begin_time'].iloc[k]
            ind_end_time = pos[i][j]['ind_end_time'].iloc[k]
    
            ind = pos[i][j]['index'].iloc[k]
            ind_floor = pos[i][j]['FloorID'].iloc[k]
            ind_building = pos[i][j]['BuildingID'].iloc[k]
            
            index_limits = np.logical_and(track[i][j][1].values >= ind_begin_time, 
                                          track[i][j][1].values < ind_end_time)
            
            track[i][j]['POSI_floor'].iloc[index_limits] = ind_floor
            track[i][j]['POSI_building'].iloc[index_limits] = ind_building

print(track[i][j].head())

      0      1       2         3        4         5        6        7  \
0  GYRO  0.018  89.931   0.58521  0.41692  -0.74526  3.00000      NaN   
1  AHRS  0.019  89.931   36.8048   5.5949 -32.75710  0.31558 -0.04448   
2  LIGH  0.020  89.940    2124.0        0       NaN      NaN      NaN   
3  ACCE  0.020  89.941  -1.38864  6.10043   7.44118  3.00000      NaN   
4  GYRO  0.031  89.951  -0.48686  0.38546   0.01344  3.00000      NaN   

         8     9  10  POSI_floor  POSI_building  
0      NaN   NaN NaN         0.0           30.0  
1 -0.25246 -43.0 NaN         0.0           30.0  
2      NaN   NaN NaN         0.0           30.0  
3      NaN   NaN NaN         0.0           30.0  
4      NaN   NaN NaN         0.0           30.0  


/usr/local/lib/python3.4/dist-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
# Pad ground truth
for i in range(len(track)):
    for j in range(len(track[i])):
        pos_pad_start = pos[i][j].iloc[0]
        pos_pad_start.at['Time'] = 0
        pos_pad_end = pos[i][j].iloc[-1]
        pos_pad_end.at['Time'] = track[i][j][1].values[-1]
        pos_array = np.vstack((pos_pad_start.values.reshape((1, pos[i][j].shape[1])), 
                                pos[i][j].values, 
                                pos_pad_end.values.reshape((1, pos[i][j].shape[1]))))
        pos[i][j] = pd.DataFrame(pos_array, columns=pos[i][j].columns.values)
print(pos[0][0].head())

     Time index          Lat       Lon FloorID BuildingID ind_begin_time  \
0       0     1  40.31347091  -3.48315       0         10              0   
1  30.341     1  40.31347091  -3.48315       0         10              0   
2  48.878     2  40.31332032  -3.48312       0         10        39.6095   
3  80.175     3  40.31328135  -3.48342       0         10        64.5265   
4  98.724     4  40.31331275  -3.48342       0         10        89.4495   

  ind_end_time  
0      39.6095  
1      39.6095  
2      64.5265  
3      89.4495  
4      117.716  


In [20]:
# Interpolate sensor time
import scipy.interpolate as interpolate

for i in range(len(track)):
    for j in range(len(track[i])):
        interp_ground_truth_lat = interpolate.interp1d(pos[i][j]['Time'].astype(float), pos[i][j]['Lat'])
        track[i][j]['interp_lat'] = interp_ground_truth_lat(track[i][j][1])
        interp_ground_truth_lon = interpolate.interp1d(pos[i][j]['Time'].astype(float), pos[i][j]['Lon'])
        track[i][j]['interp_lon'] = interp_ground_truth_lon(track[i][j][1])
print(track[0][0].head())

      0      1         2         3          4          5        6       7  \
0  ACCE  0.094  4292.788  -1.38864    4.62560    8.40844  3.00000     NaN   
1  MAGN  0.095  4292.788  -5.64000  -39.72000  -19.56000  3.00000     NaN   
2  GYRO  0.097  4292.792  -0.16432   -0.20861   -0.19609  3.00000     NaN   
3  AHRS  0.098  4292.792   28.5264     9.6349 -161.85360  0.11909 -0.2296   
4  LIGH  0.098  4292.792   14561.0          3        NaN      NaN     NaN   

        8    9  10  POSI_floor  POSI_building  interp_lat  interp_lon  
0     NaN  NaN NaN         0.0           10.0   40.313471    -3.48315  
1     NaN  NaN NaN         0.0           10.0   40.313471    -3.48315  
2     NaN  NaN NaN         0.0           10.0   40.313471    -3.48315  
3 -0.9504  3.0 NaN         0.0           10.0   40.313471    -3.48315  
4     NaN  NaN NaN         0.0           10.0   40.313471    -3.48315  


In [21]:
# create wifi data frames
wifi = []
for i in range(len(track)):
    wifi.append([])
    for j in range(len(track[i])):
        wifi[-1].append(track[i][j][[1, 4, 5, 'POSI_floor', 'POSI_building', 'interp_lat',  
                                    'interp_lon']].iloc[track[i][j][0].values == 'WIFI'])
        wifi[-1][-1].columns = ['AppTime', 'MAC', 'rssi', 'POSI_floor', 'POSI_building', 'interp_lat', 'interp_lon']
print(wifi[0][0].head())

     AppTime                MAC  rssi  POSI_floor  POSI_building  interp_lat  \
637    2.293  00:0b:86:27:36:c2 -82.0         0.0           10.0   40.313471   
638    2.293  00:0b:86:27:32:e0 -66.0         0.0           10.0   40.313471   
639    2.293  00:0b:86:27:32:e2 -66.0         0.0           10.0   40.313471   
640    2.293  00:0b:86:27:32:e1 -68.0         0.0           10.0   40.313471   
641    2.293  00:0b:86:27:36:c0 -82.0         0.0           10.0   40.313471   

     interp_lon  
637    -3.48315  
638    -3.48315  
639    -3.48315  
640    -3.48315  
641    -3.48315  


In [40]:
# split to different tracks
route_numbering = [[0, 0, 1, 1], [2, 2, 3, 3, 4, 4], [5, 6], [7, 7, 8, 8, 9]]
test_route = 3

#### total macs
742
#### building macs
0 51 ,
1 353 ,
2 158 ,
3 180 ,
#### floor macs
10 0 51 ,
10 1 0 ,
10 2 0 ,
10 3 0 ,
10 4 0 ,
10 5 0 ,
20 0 221 ,
20 1 310 ,
20 2 285 ,
20 3 0 ,
20 4 0 ,
20 5 0 ,
30 0 82 ,
30 1 66 ,
30 2 78 ,
30 3 62 ,
30 4 45 ,
30 5 125 ,
40 0 0 ,
40 1 108 ,
40 2 134 ,
40 3 123 ,
40 4 0 ,
40 5 0 ,


## Train test split should be in effect from here

In [43]:
# create list of mac addresses for all
total_macs = []
print('total macs')
for i in range(len(wifi)):
    for j in range(len(wifi[i])):
        if route_numbering[i][j] != test_route:
            track_mac_array = wifi[i][j]['MAC'].values
            for mac_add in track_mac_array:
                if not mac_add in total_macs:
                    total_macs.append(mac_add)
print(len(total_macs))

# create list of mac addresses for each building
building_macs = []
print('building macs')
for i in range(len(wifi)):
    building_macs.append([])
    for j in range(len(wifi[i])):
        if route_numbering[i][j] != test_route:
            track_mac_array = wifi[i][j]['MAC'].values
            for mac_add in track_mac_array:
                if not mac_add in building_macs[i]:
                    building_macs[i].append(mac_add)
    print(i, len(building_macs[i]), ',')

# create list of mac addresses for each building for each floor
floor_macs = {10: [[], [], [], [], [], []], 20: [[], [], [], [], [], []], 
              30: [[], [], [], [], [], []], 40: [[], [], [], [], [], []]}
for i in range(len(wifi)):
    for j in range(len(wifi[i])):
        track_mac_array = wifi[i][j]
        if route_numbering[i][j] != test_route:
            for k in range(track_mac_array.shape[0]):
                mac_line = track_mac_array.iloc[k]
                mac = mac_line['MAC']
                build = int(mac_line['POSI_building'])
                floor = int(mac_line['POSI_floor'])
                if not mac in floor_macs[build][floor]:
                    floor_macs[build][floor].append(mac)

print('floor macs')
for build in sorted(floor_macs):
    for j in range(len(floor_macs[build])):
        print(build, j, len(floor_macs[build][j]), ',')

total macs
689
building macs
0 51 ,
1 300 ,
2 158 ,
3 180 ,
floor macs
10 0 51 ,
10 1 0 ,
10 2 0 ,
10 3 0 ,
10 4 0 ,
10 5 0 ,
20 0 154 ,
20 1 244 ,
20 2 242 ,
20 3 0 ,
20 4 0 ,
20 5 0 ,
30 0 82 ,
30 1 66 ,
30 2 78 ,
30 3 62 ,
30 4 45 ,
30 5 125 ,
40 0 0 ,
40 1 108 ,
40 2 134 ,
40 3 123 ,
40 4 0 ,
40 5 0 ,


In [35]:
def dummy_wifi(wifi_df, wifi_list)

# make WiFi data into global dummy variable
wifi_dummy = []
for i in range(len(wifi)):
    wifi_dummy.append([])
    for j in range(len(wifi[i])):
        wifi_samples_cur = np.unique(wifi[i][j]['AppTime'].values)
        wifi_dummy_cur = np.ones((wifi_samples_cur.shape[0], len(building_macs[i]))) * -200
        wifi_dummy_cur = pd.DataFrame(wifi_dummy_cur, columns=building_macs[i], index=wifi_samples_cur)
        print(i, j, wifi_samples_cur.shape[0])
        for k, sample_time in enumerate(wifi_samples_cur):
            wifi_cur = wifi[i][j].iloc[wifi[i][j]['AppTime'].values == sample_time]
            for l in range(wifi_cur.shape[0]):
                wifi_dummy_cur[wifi_cur['MAC'].iat[l]].iat[k] = wifi_cur['rssi'].iat[l]
        wifi_dummy[i].append(wifi_dummy_cur)
# print(wifi_dummy[0][0].head())

0 0 232
0 1 210
0 2 222
0 3 221
1 0 150
1 1 276
1 2 188
1 3 297
1 4 73
1 5 127
2 0 274
2 1 138
3 0 131
3 1 114
3 2 98
3 3 92
3 4 128


In [36]:
# Normalize wifi and cut noise
noise_level = -50
for i in range(len(wifi_dummy)):
    for j in range(len(wifi_dummy[i])):
        max_per_line = np.amax(wifi_dummy[i][j].values, axis=1)
        wifi_dummy[i][j].iloc[:, :] = wifi_dummy[i][j].values - max_per_line.reshape((max_per_line.shape[0], 1))
        noise_not = (wifi_dummy[i][j].values >= noise_level) * 1
        wifi_dummy[i][j].iloc[:, :] = noise_not * wifi_dummy[i][j].values + (1- noise_not) * noise_level
# print(wifi_dummy[0][0].head())